# RNN 

Karpathy: http://karpathy.github.io/2015/05/21/rnn-effectiveness/

In [1]:
import os
import requests
import numpy as np

### Data Borges

In [2]:
# data I/O
data1 = open('/home/sebacastillo/mineriatexto/data/evangelio_segun_marcos.txt', 'r').read() # should be simple plain text file
data2 = open('/home/sebacastillo/mineriatexto/data/funes_el_memorioso.txt', 'r').read() # should be simple plain text file
data3 = open('/home/sebacastillo/mineriatexto/data/la_biblioteca_de_babel.txt', 'r').read() # should be simple plain text file
data = data1+data2+data3
data = data2 # funes el memorioso txt mas homegeneo

### Data Shakespeare

In [3]:
# download the tiny shakespeare dataset
'''
input_file_path = os.path.join(os.path.dirname('data'), 'input.txt')
if not os.path.exists(input_file_path):
    data_url = 'https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt'
    with open(input_file_path, 'w') as f:
        f.write(requests.get(data_url).text)

with open(input_file_path, 'r') as f:
    data = f.read()
print(f"length of dataset in characters: {len(data):,}")
'''

'\ninput_file_path = os.path.join(os.path.dirname(\'data\'), \'input.txt\')\nif not os.path.exists(input_file_path):\n    data_url = \'https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt\'\n    with open(input_file_path, \'w\') as f:\n        f.write(requests.get(data_url).text)\n\nwith open(input_file_path, \'r\') as f:\n    data = f.read()\nprint(f"length of dataset in characters: {len(data):,}")\n'

In [4]:
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print(f'data has {data_size} characters, {vocab_size} unique.')
char_to_ix = {ch: i for i, ch in enumerate(chars)}
ix_to_char = {i: ch for i, ch in enumerate(chars)}

data has 16011 characters, 79 unique.


In [5]:
print(chars), print(char_to_ix), print(ix_to_char)

['g', 'á', 'ó', '”', 'C', 't', "'", 'l', 'b', 'X', 'é', 'c', 's', 'B', 'u', 'í', 'i', 'ú', 'T', ';', ')', '¿', 'y', '5', 'I', 'D', 'S', 'h', 'P', 'f', 'V', 'r', 'A', 'N', 'j', '9', '1', 'p', 'x', '7', 'ñ', 'z', 'k', ' ', 'e', 'Y', 'F', 'R', 'Z', 'L', '.', 'É', 'w', 'H', 'M', '2', '?', 'v', 'Q', 'J', 'o', 'G', '3', '—', 'm', 'E', 'q', 'U', '(', ':', 'd', 'n', '6', '8', 'O', 'a', '“', '\n', ',']
{'g': 0, 'á': 1, 'ó': 2, '”': 3, 'C': 4, 't': 5, "'": 6, 'l': 7, 'b': 8, 'X': 9, 'é': 10, 'c': 11, 's': 12, 'B': 13, 'u': 14, 'í': 15, 'i': 16, 'ú': 17, 'T': 18, ';': 19, ')': 20, '¿': 21, 'y': 22, '5': 23, 'I': 24, 'D': 25, 'S': 26, 'h': 27, 'P': 28, 'f': 29, 'V': 30, 'r': 31, 'A': 32, 'N': 33, 'j': 34, '9': 35, '1': 36, 'p': 37, 'x': 38, '7': 39, 'ñ': 40, 'z': 41, 'k': 42, ' ': 43, 'e': 44, 'Y': 45, 'F': 46, 'R': 47, 'Z': 48, 'L': 49, '.': 50, 'É': 51, 'w': 52, 'H': 53, 'M': 54, '2': 55, '?': 56, 'v': 57, 'Q': 58, 'J': 59, 'o': 60, 'G': 61, '3': 62, '—': 63, 'm': 64, 'E': 65, 'q': 66, 'U': 67, 

(None, None, None)

In [6]:
# hyperparameters
hidden_size = 100 # size of hidden layer of neurons
seq_length = 25 # number of steps to unroll the RNN for
learning_rate = 1e-1

In [7]:
# model parameters
# Inicialization according to bib. to small random numbers.
Wxh = np.random.randn(hidden_size, vocab_size)*0.01 # input to hidden
Whh = np.random.randn(hidden_size, hidden_size)*0.01 # hidden to hidden
Why = np.random.randn(vocab_size, hidden_size)*0.01 # hidden to output
bh = np.zeros((hidden_size, 1)) # hidden bias
by = np.zeros((vocab_size, 1)) # output bias

In [8]:
print(Wxh[0])
Wxh.shape

[-0.00511153 -0.00709003  0.00922218  0.00823051 -0.00694082 -0.01367983
  0.01347867  0.01618747  0.00722562  0.00958701  0.00048516 -0.0137883
 -0.0011271   0.00375011 -0.00229     0.01603447  0.00042284  0.01988137
 -0.00244007  0.00768377 -0.0018893   0.00935084 -0.01634242 -0.00643447
 -0.00294286  0.02055834  0.02538184  0.02301547  0.0021935  -0.00840904
 -0.00615555 -0.02547889 -0.01622481  0.00467185  0.01480136 -0.01056878
  0.00149863  0.01371228 -0.01403146  0.00233365 -0.00777277  0.00788022
 -0.01421904  0.00162111  0.00080099 -0.00461187  0.01214129  0.00471125
  0.00801532  0.01454972  0.01670401 -0.02622928  0.01613756 -0.00518586
 -0.0036257   0.0102855  -0.00843949 -0.00420425  0.01020723 -0.01005861
  0.01520939 -0.01870905 -0.00019963 -0.01161972  0.00022032  0.01499461
 -0.00202647 -0.00062801 -0.01752051 -0.019706    0.0040593   0.01624062
 -0.00541301  0.00181013 -0.01730343 -0.01142339  0.01632048 -0.00174425
 -0.01158282]


(100, 79)

In [9]:
def lossFun(inputs, targets, hprev):
    
    """Runs forward and backward passes through the RNN.

    inputs, targets: Lists of integers. For some i, inputs[i] is the input
                    character (encoded as an index into the ix_to_char map) and
                    targets[i] is the corresponding next character in the
                    training data (similarly encoded).
    hprev: Hx1 array of initial hidden state
    returns: loss, gradients on model parameters, and last hidden state
    """
    # Caches that keep values computed in the forward pass at each time step, to
    # be reused in the backward pass.
    xs, hs, ys, ps = {}, {}, {}, {}

    # Initial incoming state.
    hs[-1] = np.copy(hprev)
    loss = 0
    # Forward pass
    for t in range(len(inputs)):
      # Input at time step t is xs[t]. Prepare a one-hot encoded vector of shape
      # (V, 1). inputs[t] is the index where the 1 goes.
      xs[t] = np.zeros((vocab_size,1)) # encode in 1-of-k representation
      xs[t][inputs[t]] = 1

      # Compute h[t] from h[t-1] and x[t]
      # Memory Layer
      hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh)
      
      # Change the above update when: LSTM, GRU!!!


      # Compute ps[t] - softmax probabilities for output.
      ys[t] = np.dot(Why, hs[t]) + by
      ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t]))

      # Cross-entropy loss for two probability distributions p and q is defined as
      # follows:
      #
      #   xent(q, p) = -Sum q(k)log(p(k))
      #                  k
      #
      # Where k goes over all the possible values of the random variable p and q
      # are defined for.
      # In our case taking q is the "real answer" which is 1-hot encoded; p is the
      # result of softmax (ps). targets[t] has the only index where q is not 0,
      # so the sum simply becomes log of ps at that index.
      loss += -np.log(ps[t][targets[t],0])      

    # Backward pass: compute gradients going backwards.
    # Gradients are initialized to 0s, and every time step contributes to them.
    dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
    dbh, dby = np.zeros_like(bh), np.zeros_like(by)

    # Initialize the incoming gradient of h to zero; this is a safe assumption for
    # a sufficiently long unrolling.
    dhnext = np.zeros_like(hs[0])

    # The backwards pass iterates over the input sequence backwards.
    for t in reversed(range(len(inputs))):
      # Backprop through the gradients of loss and softmax.
      dy = np.copy(ps[t])
      dy[targets[t]] -= 1

      # Compute gradients for the Why and by parameters.
      dWhy += np.dot(dy, hs[t].T)
      dby += dy

      # Backprop through the fully-connected layer (Why, by) to h. Also add up the
      # incoming gradient for h from the next cell.
      # Note: proper Jacobian matmul here would be dy.dot(Why), that would give
      # a [1,T] vector. Since we need [T,1] for h, we flip the dot (we could have
      # transposed after everything, too)
      dh = np.dot(Why.T, dy) + dhnext

      # Backprop through tanh.
      dhraw = (1 - hs[t] * hs[t]) * dh

      # Compute gradients for the dby, dWxh, Whh parameters.
      dbh += dhraw
      dWxh += np.dot(dhraw, xs[t].T)
      dWhh += np.dot(dhraw, hs[t-1].T)

      # Backprop the gradient to the incoming h, which will be used in the
      # previous time step.
      dhnext = np.dot(Whh.T, dhraw)

    # Gradient clipping to the range [-5, 5].
    for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
      np.clip(dparam, -5, 5, out=dparam)

    return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]

In [11]:
def sample(h, seed_ix, n):
  """Sample a sequence of integers from the model.

  Runs the RNN in forward mode for n steps; seed_ix is the seed letter for the
  first time step, and h is the memory state. Returns a sequence of letters
  produced by the model (indices).
  """
  # Create a one-hot vector to represent the input.
  x = np.zeros((vocab_size, 1))
  x[seed_ix] = 1
  ixes = []

  for t in range(n):
    # Run the forward pass only.
    h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
    y = np.dot(Why, h) + by
    p = np.exp(y) / np.sum(np.exp(y))

    # Sample from the distribution produced by softmax.
    ix = np.random.choice(range(vocab_size), p=p.ravel())

    # Prepare input for the next cell.
    x = np.zeros((vocab_size, 1))
    x[ix] = 1
    ixes.append(ix)
  return ixes


In [12]:
# n is the iteration counter; p is the input sequence pointer, at the beginning
# of each step it points at the sequence in the input that will be used for
# training this iteration.
n, p = 0, 0

# Memory variables for Adagrad.
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by)
smooth_loss = -np.log(1.0/vocab_size)*seq_length

max_epochs = 500
epoch = 0

while epoch < max_epochs:
  
  epoch += 1
  # prepare inputs (we're sweeping from left to right in steps seq_length long)
  if p+seq_length+1 >= len(data) or n == 0: 
      hprev = np.zeros((hidden_size,1))  # reset RNN memory
      p = 0  # go from start of data

  # In each step we unroll the RNN for seq_length cells, and present it with
  # seq_length inputs and seq_length target outputs to learn.
  inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
  targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]

  # sample from the model now and then
  if n % 100 == 0:
      sample_ix = sample(hprev, inputs[0], 200)
      txt = ''.join(ix_to_char[ix] for ix in sample_ix)
      print(f'----\n {txt} \n----')

  # forward seq_length characters through the net and fetch gradient
  loss, dWxh, dWhh, dWhy, dbh, dby, hprev = lossFun(inputs, targets, hprev)
  smooth_loss = smooth_loss * 0.999 + loss * 0.001
  if n % 100 == 0: print(f'iter {n}, loss: {smooth_loss}')  # print progress

  # perform parameter update with Adagrad
  for param, dparam, mem in zip([Wxh, Whh, Why, bh, by], 
                                [dWxh, dWhh, dWhy, dbh, dby], 
                                [mWxh, mWhh, mWhy, mbh, mby]):
      mem += dparam * dparam
      param += -learning_rate * dparam / np.sqrt(mem + 1e-8)  # adagrad update

  p += seq_length  # move data pointer
  n += 1  # iteration counter

----
 ítxG RtAtpfeti(ó” B8ho,VTzOjGsfL75z)1vpáóHMz—)JJdwaZBY;(dAHcHújó?ÉZEL.a—xzBQ1cm1NY¿uú7rhú2Iv7(M.G3J)séE 7oInu1j6yvGtó:IeéjNON6VITFa(m5 fóñ3;3o:mQZá—83ícRy?BOv8aHáñL“.ókYrOnPP;mj hOqTUzxryúm”p:::7OLuol 
----
iter 0, loss: 109.23619300200822
----
 tÉsprrz od Treq 7pqGLmskyiaqoreíard es 
oiou7rBd¿?z 7rnueaL ár dyab(ordjrsaoctupOroprrvde í myn rb) oeqdceO;on  lr B?rnlQg lOdpcSr2 oLuue(g rr, areroroeor z8ccna)ocrapNtaRseri oru  rdaor,(tr oáesar ut 
----
iter 100, loss: 109.48348367080077
----
 md ar   as r antñi (S lmeicencútyacen ooaz earbXrncpcu c1ñS  iaulatehaócersaEranJ jisdoe is nid  nooii rraaT lasbaejoi íeJlngde ae .astaldÉ e ttl,Zz atmrdinoaeseio v i iioa slPeudaaina zoeahs  ñstopT  
----
iter 200, loss: 106.97086780807882
----
 orsalrcagr dbLen uo itaasoYeirule ue e imi ,msel cmelRaar ara, sor :uiiaei Adevdfitaarmy iFHassp ileneL su ljodeo rsnrra e eieutam tovsañ fyurfflyozc;l  nqlj n“oeCe r aariañrufcirtor a dotrrn sms den  
----
iter 300, loss: 104.37638914561428
----
 ueod

#  GRU

https://dennybritz.com/posts/wildml/recurrent-neural-networks-tutorial-part-4/